In [1]:
from gymnasium.wrappers import FrameStackObservation, AtariPreprocessing
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from collections import deque
from datetime import datetime
from pathlib import Path
import gymnasium as gym
import numpy as np
import random
import ale_py
import time
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

In [2]:
import sys
npath = str(Path.cwd().parents[0])  # [0] = ../  [1] = ../../
sys.path.append(npath)
from stuff import *
sys.path.remove(npath)

In [3]:
os.makedirs("checkpoints", exist_ok=True)
os.makedirs("runs", exist_ok=True)

In [4]:
print(gym.__version__, torch.__version__, torch.get_num_threads(), torch.get_num_interop_threads(), sep='\n')

1.1.0
2.6.0+cpu
10
10


In [5]:
class QNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, ...)
                                              # input (m, 4,84,84) where m=batch_size
        self.conv1 = nn.Conv2d(4, 32, 8, 4)   # -> relu (m, 32,20,20)
        self.conv2 = nn.Conv2d(32, 64, 4, 2)  # -> relu (m, 64,9,9)
        self.conv3 = nn.Conv2d(64, 64, 3, 1)  # -> relu (m, 64,7,7) -> flatten (m, 3136)
        # nn.Linear(in_features, out_features, ...)
        self.fc1 = nn.Linear(3136, 512)  # -> relu (m, 512)
        self.fc2 = nn.Linear(512, 4)  # -> output (m, 4)
        
    def forward(self, x):
        # x = x / 255.0  # should we scale to [0, 1]
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)  # flatten it to (m, 3136)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # no output activation
        return x

In [6]:
def make_env(render_mode=None):
    global env
    if 'env' in globals(): 
        env.reset()
        env.close()
        del env
    env = gym.make("BreakoutNoFrameskip-v4", render_mode=render_mode)
    env = AtariPreprocessing(env)
    env = FrameStackObservation(env, 4)
    return env.reset()

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"using device: {device}")

using device: cpu


In [8]:
online_net = QNetwork().to(device)
target_net = QNetwork().to(device)
target_net.eval()

def sync(): target_net.load_state_dict(online_net.state_dict())
sync()

episode_count, step_count = 0, 0

In [9]:
proj_name = 'breakout-03-05'

checkpoint_version = 0
prev_mem_name = '\0'
def load_checkpoint(checkpoint_file):
    global checkpoint_version, episode_count, step_count
    checkpoint = torch.load('./checkpoints/' + checkpoint_file, weights_only=False, map_location=torch.device(device))
    online_net.load_state_dict(checkpoint['online_state_dict'])
    target_net.load_state_dict(checkpoint['target_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    episode_count, step_count = checkpoint.get('episode_count'), checkpoint.get('step_count')
    checkpoint_version = checkpoint.get('checkpoint_version')
    checkpoint_version += 1
    no_mem = False
    try: memory = torch.load('./checkpoints/mem-' + checkpoint_file, weights_only=False)
    except: no_mem = True
    print('loaded', checkpoint_file, 'memory file could not be accessed' if no_mem else '')

def save_checkpoint():
    global prev_mem_name, checkpoint_version
    name = f'./checkpoints/{checkpoint_version}-{proj_name}-{episode_count}e-{step_count}s.pth'
    mem_name = f'./checkpoints/mem-{checkpoint_version}-{proj_name}-{episode_count}e-{step_count}s.pth'
    checkpoint = {
        'online_state_dict': online_net.state_dict(),
        'target_state_dict': target_net.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'step_count': step_count,
        'episode_count': episode_count,
        'checkpoint_version': checkpoint_version
    }
    torch.save(checkpoint, name)
    # torch.save(memory, mem_name)  # saving memory takes a bit of time
    checkpoint_version += 1
    if os.path.exists(prev_mem_name): os.remove(prev_mem_name)
    prev_mem_name = mem_name
    print('saved', name)

# Modified hyperparameters from Nature

In [10]:
batch_size = 32
memory = ReplayBuffer(100_000)  # holding 1 million transitions requires ~4*84*84*2*1e6 ~ 56 gb of memory, 100k should be enough
sync_freq = 10_000  # environment steps
gamma = 0.99
learn_freq = 4
learning_rate = 0.00025
eps_max = 1.0  # initial epsilon
eps_min = 0.1  # final
eps_anneal_steps = 1_000_000  # linearly anneal from eps_max to eps_min over X steps
learning_starts = 50_000  # current policy is run for X steps before learning starts
reward_clip = (-999, 999)
max_steps = 10000
max_norm = 1.0

optimizer = optim.AdamW(online_net.parameters(), lr=learning_rate)
criterion = nn.HuberLoss(delta=1.0)

In [11]:
def get_epsilon():  # epsilon schedule
    effective_steps = step_count
    return max(eps_min, eps_min + (eps_max - eps_min) * (1 - effective_steps / eps_anneal_steps))

def do_action(action, update_memory=True):
    global state
    action = epsilon_greedy(state)
    next_state, reward, terminated, truncated, info = env.step(action)
    reward = np.clip(reward, *reward_clip)
    if update_memory: memory.push(state, action, reward, next_state, terminated)
    state = next_state
    return reward, terminated, truncated
    
def greedy(s):
    with torch.no_grad():
        return online_net(torch.tensor(s, dtype=torch.float, device=device).unsqueeze(0)).argmax().item()
    
def epsilon_greedy(s): 
    return env.action_space.sample() if np.random.random() < get_epsilon() else greedy(s)

In [18]:
os.listdir('./checkpoints')

['0-breakout-03-05-1001e-185283s.pth',
 '1-breakout-03-05-1483e-290232s.pth',
 '10-breakout-03-05-7092e-3343516s.pth',
 '11-breakout-03-05-8092e-3979952s.pth',
 '3-breakout-03-05-1565e-311008s.pth',
 '4-breakout-03-05-2566e-613672s.pth',
 '5-breakout-03-05-3566e-1137988s.pth',
 '6-breakout-03-05-3743e-1240756s.pth',
 '7-breakout-03-05-4091e-1445352s.pth',
 '8-breakout-03-05-5092e-2065169s.pth',
 '9-breakout-03-05-6092e-2731172s.pth']

In [13]:
load_checkpoint('11-breakout-03-05-8092e-3979952s.pth')

loaded 11-breakout-03-05-8092e-3979952s.pth memory file could not be accessed


In [14]:
print(f'checkpoint_version {checkpoint_version}')
print(f'episode_count {episode_count}')
print(f'step_count {step_count}')

checkpoint_version 12
episode_count 8092
step_count 3979952


In [17]:
train_episodes = 1_000_000

writer = SummaryWriter(log_dir='./runs')
return_history = deque(maxlen=100)
make_env(None)
start_episode, start_step = episode_count, step_count

try:
    t0 = time.time()
    # while len(memory) < learning_starts:
    while len(memory) < batch_size:
        state, info = env.reset()
        for step in range(max_steps):
            reward, terminated, truncated = do_action(env.action_space.sample())
            if len(memory) % 100 == 0:
                clear_output(wait=True)
                print(f'collecting initial training samples {len(memory)}/{learning_starts} ({time.time() - t0:.2f} s)')
            if terminated or truncated:
                break

    episode_times = deque(maxlen=5)
    step_times = deque(maxlen=5)
    t0 = time.time()
    t1 = time.time()
    for episode in range(train_episodes):
        state, info = env.reset()
        episode_return = 0
        episode_count += 1

        for step in range(max_steps):
            action = epsilon_greedy(state)
            reward, truncated, terminated = do_action(action)
            episode_return += reward
            step_count += 1

            if step_count % sync_freq == 0:  # update target network
                sync()

            if step_count % learn_freq == 0:  # update online net
                states, actions, rewards, next_states, terminateds = memory.sample(batch_size)  # as_tensor does not copy
                states = torch.as_tensor(states, dtype=torch.float, device=device)            # (m, 4,84,84)
                next_states = torch.as_tensor(next_states, dtype=torch.float, device=device)  # (m, 4,84,84)
                actions = torch.as_tensor(actions, dtype=torch.long, device=device).reshape(-1, 1)           # (m, 1)
                rewards = torch.as_tensor(rewards, dtype=torch.float, device=device).reshape(-1, 1)          # (m, 1)
                terminateds = torch.as_tensor(terminateds, dtype=torch.float, device=device).reshape(-1, 1)  # (m, 1)

                pred = online_net(states).gather(1, actions)  # predicted Q-values of the selected action

                with torch.no_grad():
                    y = rewards + gamma * target_net(next_states).max(axis=1, keepdim=True).values * (1 - terminateds)

                loss = criterion(pred, y)  # don't need to detach since y.requires_grad is False

                optimizer.zero_grad()
                loss.backward()
                total_norm = torch.nn.utils.clip_grad_norm_(online_net.parameters(), max_norm)
                writer.add_scalar('loss', loss.item(), step_count)
                writer.add_scalar('total_norm', total_norm, step_count)
                optimizer.step()

            if terminated or truncated:
                break

        writer.add_scalar('episode_steps', step, step_count)
        writer.add_scalar('episode_return', episode_return, step_count)
        writer.add_scalar('epsilon', get_epsilon(), step_count)
        return_history.append(episode_return)

        t = time.time() - t1
        episode_times.append(1/t)
        step_times.append(step/t)
        t1 = time.time()

        if episode % 1000 == 0 and episode != 0:
            save_checkpoint()
            writer.flush()

        info_freq = 10
        if episode % info_freq == 0 or episode == train_episodes-1:
            tt = time.time() - t0
            et = episode_count - start_episode
            st = step_count - start_step
            # plt.plot(return_history)

            clear_output(wait=True)
            print(f'episode {et}\t({np.mean(episode_times):.2f}/s)  [total {episode_count}]')
            print(f'step {st}\t({np.mean(step_times):.1f}/s)  [total {step_count}]')
            print(f'time {tt:.2f} s')
            print('---')
            print(f'avg. return: {np.mean(return_history):.5f}  (last 100 episodes)')
            print(f'epsilon {get_epsilon():.5f}')
            # plt.show()

except KeyboardInterrupt:
    print('\nkeyboard interrupt - cleaning up...')
finally:
    if episode_count - start_episode > 20:
        save_checkpoint()
    writer.close()
    env.close()
    print('done.')

episode 11	(0.03/s)  [total 8118]
step 7345	(22.9/s)  [total 3995617]
time 330.51 s
---
avg. return: 19.27273  (last 100 episodes)
epsilon 0.10000

keyboard interrupt - cleaning up...
done.


In [16]:
# benchmark
# = baseline : 22.9 steps/s
# + as_tensor: 23.3 steps/s
# + async env: ???  steps/s